<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100"/>

# Cours TAL - Laboratoire 6
# Trois méthodes de désambiguïsation lexicale

**Objectif**

L'objectif de ce laboratoire est d'implémenter et de comparer plusieurs méthodes de désambiguïsation lexicale (en anglais, *Word Sense Disambiguation* ou WSD).  Vous utiliserez un corpus avec plusieurs milliers de phrases, chaque phrase contenant une occurrence du mot anglais *interest* annotée avec le sens que ce mot possède dans la phrase respective.  Les trois méthodes sont les suivantes (elles seront détaillées par la suite) :

* Algorithme de Lesk simplifié.
* Utilisation de word2vec.
* Classification supervisée utilisant des traits lexicaux.

Les deux premières méthodes n'utilisent pas l'apprentissage automatique.  Elles fonctionnent selon le même principe : comparer le contexte d'une occurrence de *interest* avec chacune des définitions des sens (*synsets*) et choisir la définition la plus proche du contexte.  L'algorithme de Lesk définit la proximité comme le nombre de mots en commun, alors que word2vec la calcule comme la similarité de vecteurs.  La dernière méthode vise à classifier les occurrences de *interest*, les sens étant les classes, et les attributs étant les mots du contexte (apprentissage supervisé).

## 1. Analyse des données

Téléchargez le corpus *interest* depuis le [site du Prof. Ted Pedersen](http://www.d.umn.edu/~tpederse/data.html) (il se trouve en bas de sa page web).  Téléchargez l'archive ZIP marquée *original format without POS tags* et extrayez le fichier `interest-original.txt`.  Téléchargez également le fichier `README.int.txt` indiqué à la ligne au-dessus. Veuillez répondre brièvement aux questions suivantes :

a. Quelles sont les URL du fichier ZIP et celle du fichier `README.int.txt` ?

b. Quel est le format du fichier `interest-original.txt` et comment sont annotés les sens de *interest* ?

c. Est-ce qu'il y a aussi des occurrences au pluriel (*interests*) à traite ?

d. Comment sont annotées les phrases qui contiennent plusieurs occurrences du mot *interest* ?

# Veuillez répondre ici (en commentaire) aux questions.
a.
- fichier ZIP: https://www.d.umn.edu/~tpederse/Data/interest-original.nopos.tar.gz
- README: https://www.d.umn.edu/~tpederse/Data/README.int.txt

b.
Voici les caractéristiques du fichier `interest-original.txt`:
- Les phrases sont séparées par `$$`.
- Les phrases ont déjà été tokenisées.
- Le mot `interest` est remplacé par `interest_num`, où le numéro peut être de 1 à 6 et
représente le sens du mot `interest` dans la phrase. Les 6 sens possibles sont présentés
à la question e.
- Les mots `interests` au pluriel ont simplement un `s` en plus avant le numéro.

c.

Oui, il y aussi des occurrences au pluriel à traiter. Lorsque le mot `interest` est au pluriel,
il y a simplement un `s` à la fin du mot, avant le numéro. Par exemple, on peut avoir `interests_5`
pour un mot `interest` au pluriel.

d. Par exemple, en regardant le document, on peut voir par exemple cette phrase apparaissant deux fois dans le fichier, contenant deux fois le mot interest.
```
====================================== as  part  of  its decision  to leave  the building  ,  chase  tried to sell  its interest_5  , along with  the atlas estate 's *interest  , shortly after  the october 1987 stock market crash  .
$$
====================================== as  part  of  its decision  to leave  the building  ,  chase  tried to sell  its *interest  , along with  the atlas estate 's interest_5  , shortly after  the october 1987 stock market crash  .
$$
```
On peut voir que lorsqu'une phrase contient plusieurs occurrences du mot `interest`, seulement un de ces mots est annoté et les autres sont préfixés d'un `*`. La phrase est dupliquée avec, chaque fois, une annotation sur un des `interest` différents.


**1e.** D'après le fichier `README.int.txt`, quelles sont les définitions des six sens de *interest* annotés dans les données et quelles sont leurs fréquences ? Vous pouvez copier/coller l'extrait de `README`ici.

### Veuillez répondre ici (en commentaire) à la question.

```
Sense 1 =  361 occurrences (15%) - readiness to give attention
Sense 2 =   11 occurrences (01%) - quality of causing attention to be given to
Sense 3 =   66 occurrences (03%) - activity, etc. that one gives attention to
Sense 4 =  178 occurrences (08%) - advantage, advancement or favor
Sense 5 =  500 occurrences (21%) - a share in a company or business
Sense 6 = 1252 occurrences (53%) - money paid for the use of money
```


**1f.** De quel dictionnaire viennent les sens précédents ? Où peut-on le consulter en ligne ?  Veuillez aligner les définitions du dictionnaire avec les six sens annotés en écrivant par exemple `Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"`.

### Veuillez répondre ici (en commentaire) à la question.
Les sens sont tirés du dictionnaire Longman's Dictionary of Contemporary English qui est disponible au lien suivant : https://www.ldoceonline.com/ et les sens du mot `interest` peuvent être trouvés à ce lien : https://www.ldoceonline.com/dictionary/interest

- Sense 1 = "if you have an interest in something or someone, you want to know or learn more about them"
- Sense 2 = "a quality or feature of something that attracts your attention or makes you want to know more about it"
- Sense 3 = "an activity that you enjoy doing or a subject that you enjoy studying"
- Sense 4 = "the things that bring advantages to someone or something"
- Sense 5 = "if you have an interest in a particular company or industry, you own shares in it"
- Sense 6 = "money paid to you by a bank or financial institution when you keep money in an account there"


**1g.** En consultant [WordNet en ligne](http://wordnetweb.princeton.edu/perl/webwn), trouvez les définitions des synsets  pour le **nom commun** *interest*.  Combien de synsets y a-t-il ?  Veuillez indiquer comme avant la **définition** de chaque synset pour chacun des six sens ci-dessus (au besoin, fusionner ou ignorer des synsets).

### Veuillez répondre ici (en commentaire) à la question.

Il y a 7 synsets dans lequel le mot `interest` apparait dans WordNet.
1. interest, involvement -- (a sense of concern with and curiosity about someone or something; "an interest in music")
2. interest, interestingness -- (the power of attracting or holding one's attention (because it is unusual or exciting etc.); "they said nothing of great interest"; "primary colors can add interest to a room")
3. sake, interest -- (a reason for wanting something done; "for your sake"; "died for the sake of his country"; "in the interest of safety"; "in the common interest")
4. interest -- (a fixed charge for borrowing money; usually a percentage of the amount borrowed; "how much interest do you pay on your mortgage?")
5. pastime, interest, pursuit -- (a diversion that occupies one's time and thoughts (usually pleasantly); "sailing is her favorite pastime"; "his main pastime is gambling"; "he counts reading among his interests"; "they criticized the boy for his limited pursuits")
6. interest, stake -- ((law) a right or legal share of something; a financial involvement with something; "they have interests all over the world"; "a stake in the company's future")
7. interest, interest group -- ((usually plural) a social group whose members control some field of activity and who have common aims; "the iron interests stepped up production")

Les sens donnés dans le fichier correspondent aux synsets suivant:

| Sens du fichier `interest-original.txt` | Synset WordNet |
|:---------------------------------------:|:--------------:|
|                    1                    |       1        |
|                    2                    |       2        |
|                    3                    |       5        |
|                    4                    |       3        |
|                    5                    |       6        |
|                    6                    |       4        |

Le synset 7 n'est pas représenté.

**1h.** Définissez (manuellement, ou avec quelques lignes de code) une liste nommée `senses1` avec les mots des définitions du README, en supprimant les stopwords (p.ex. les mots < 4 lettres).  Affichez la liste.

In [2]:
import nltk
from nltk import NaiveBayesClassifier

In [3]:
# Veuillez répondre ici à la question et créer la variable 'senses1' (liste de 6 listes de chaînes).
senses1 = [['readiness', 'give', 'attention'], ['quality', 'causing', 'attention'], ['activity', 'gives', 'attention'], ['advantage', 'advancement', 'favor'], ['share', 'company', 'business'], ['paid', 'money']]

print(senses1)

[['readiness', 'give', 'attention'], ['quality', 'causing', 'attention'], ['activity', 'gives', 'attention'], ['advantage', 'advancement', 'favor'], ['share', 'company', 'business'], ['paid', 'money']]


**1i.** En combinant les définitions obtenues aux points (4) et (5) ci-dessus, construisez une liste nommée `senses2` avec pour chacun des sens de *interest* une liste de **mots-clés** correspondants.  Vous pouvez concaténer les définitions, puis écrire des instructions en Python pour extraire les mots (uniques).  Respectez l'ordre des sens données par `README`, et à la fin affichez `senses2`.

In [4]:
# Veuillez répondre ici à la question,  et créer la variable 'senses2' (liste de 6 listes de chaînes).
senses2 = [['interest', 'something', 'someone', 'know', 'learn', 'about', 'sense', 'concern', 'curiosity'],
    ['quality', 'feature', 'something', 'attracts', 'attention', 'makes', 'want', 'know', 'about', 'power', 'attracting', 'holding'],
    ['activity', 'enjoy', 'subject', 'studying', 'diversion', 'occupies', 'time', 'thoughts'],
    ['things', 'bring', 'advantages', 'someone', 'something', 'reason', 'wanting', 'something'],
    ['interest', 'particular', 'company', 'industry', 'own', 'shares', 'law', 'right', 'legal', 'share', 'something', 'financial', 'involvement', 'something'],
    ['money', 'paid', 'bank', 'financial', 'institution', 'keep', 'account', 'fixed', 'charge', 'borrowing', 'usually', 'percentage', 'amount', 'borrowed']
]

print(senses2)
# Nous avons décidé de construire cette liste à la main et avons retiré un certains nombre de stopwords

[['interest', 'something', 'someone', 'know', 'learn', 'about', 'sense', 'concern', 'curiosity'], ['quality', 'feature', 'something', 'attracts', 'attention', 'makes', 'want', 'know', 'about', 'power', 'attracting', 'holding'], ['activity', 'enjoy', 'subject', 'studying', 'diversion', 'occupies', 'time', 'thoughts'], ['things', 'bring', 'advantages', 'someone', 'something', 'reason', 'wanting', 'something'], ['interest', 'particular', 'company', 'industry', 'own', 'shares', 'law', 'right', 'legal', 'share', 'something', 'financial', 'involvement', 'something'], ['money', 'paid', 'bank', 'financial', 'institution', 'keep', 'account', 'fixed', 'charge', 'borrowing', 'usually', 'percentage', 'amount', 'borrowed']]


**1j.** Chargez les données depuis `interest-original.txt` dans une liste appelée `sentences` qui contient pour chaque phrase la liste des mots (sans les séparateurs *$$* et *===...*).  Ces phrases sont-elles déjà tokenisées en mots ?  Sinon, faites-le.  À ce stade, ne modifiez pas encore les occurrences annotées *interest(s)\_X*.  Comptez le nombre total de phrases et affichez-en trois au hasard.

In [5]:
# Veuillez répondre ici à la question.
sentences = []
UNWANTED_CHAR = ['$$', '=']

with open('data/interest-original.txt', 'r') as f:
    for line in f:
        line = line.strip()
        sentence = []
        for word in line.split():
            if not any(char in word for char in UNWANTED_CHAR):
                sentence.append(word)
        if len(sentence) > 0:
            sentences.append(sentence)

print(f"Il y a {len(sentences)} phrases.\nEn voici 3 au hasard :")
print(sentences[151:154])

Il y a 2368 phrases.
En voici 3 au hasard :
[['investor', 'interest_1', 'in', 'stock', 'funds', '``', 'has', "n't", 'stalled', 'at', 'all', ',', "''", 'mr.', 'hines', 'maintains', '.'], ["''", 'it', 'is', 'in', 'the', 'western', 'interest_4', 'to', 'see', 'mr.', 'gorbachev', 'succeed', '.'], ['revco', 'insists', 'that', 'the', 'proposal', 'is', 'simply', 'an', '``', 'expression', 'of', 'interest_1', ',', "''", 'because', 'under', 'chapter', '11', 'revco', 'has', '``', 'exclusivity', 'rights', "''", 'until', 'feb.', '28', '.']]


### Réponse à la question

Oui, les phrases sont déjà tokenisées en mots. Il faut toutefois retirer les séparateurs.

## 2. Algorithme de Lesk simplifié

**2a.** Définissez une fonction `wsd_lesk(senses, sentence)` qui prend deux arguments : une liste de listes de mots-clés (comme `senses1` et `senses2` ci-dessus) et une phrase avec une occurrence annotée de *interest* ou *interests*, et qui retourne l'index du sens le plus probable (entre 1 et 6) selon l'algorithme de Lesk.  Cet algorithme choisit le sens qui a le maximum de mots en commun avec le contexte de *interest*.  Vous pouvez choisir vous-mêmes la taille de ce voisinage (`window_size`).  En cas d'égalité entre deux sens, tirer la réponse au sort.

In [6]:
def wsd_lesk(senses, sentence, window_size):
    index = -1
    pos = 0
    for word in sentence:
        if "interest" in word:
            index = pos
            break
        pos += 1
    context = sentence[max(index-window_size, 0):min(index+window_size, len(sentence))]
    intersections = []
    for sense in senses:
        intersections.append(len(set(context).intersection(set(sense))))
    return intersections.index(max(intersections)) + 1

**2b.** Définissez maintenant une fonction `evaluate_wsd(fct_name, senses, sentences)` qui prend en paramètre le nom de la méthode de similarité (pour commencer : `wsd_lesk`) ainsi que la liste des mots-clés par sens, et la liste de phrases, et qui retourne le score de la méthode de similarité.  Ce score sera tout simplement le pourcentage de réponses correctes (sens trouvé identique au sens annoté).

In [7]:
import re

def evaluate_wsd(fct_name, senses, sentences, window_size = 7):
  correct_classifications = 0
  for sentence in sentences:
    for index, word in enumerate(sentence):
      if re.match('^interest[s]?_[1-6]$', word): # regex qui match les occurrences annotées
        real_sense = int(word[-1])
        found_sense = fct_name(senses, sentence, window_size)
        if real_sense == found_sense:
          correct_classifications += 1
  return correct_classifications / len(sentences)

evaluate_wsd(wsd_lesk, senses2, sentences, 5)

0.19890202702702703

**2c.** En fixant au mieux la taille de la fenêtre autour de *interest*, quel est le meilleur score de la méthode de Lesk simplifiée ?  Quelle liste de sens conduit à de meilleurs scores, `senses1` ou `senses2` ?

In [8]:
# Veuillez répondre ici à la question.

print('Sense1')
for i in range(1, 20):
  print(f'Score pour une taille de fenêtre de {i}: {evaluate_wsd(wsd_lesk, senses1, sentences, i)}')

print('\nSense2')
for i in range(1, 20):
  print(f'Score pour une taille de fenêtre de {i}: {evaluate_wsd(wsd_lesk, senses2, sentences, i)}')

Sense1
Score pour une taille de fenêtre de 1: 0.15371621621621623
Score pour une taille de fenêtre de 2: 0.16258445945945946
Score pour une taille de fenêtre de 3: 0.16807432432432431
Score pour une taille de fenêtre de 4: 0.1710304054054054
Score pour une taille de fenêtre de 5: 0.17567567567567569
Score pour une taille de fenêtre de 6: 0.18074324324324326
Score pour une taille de fenêtre de 7: 0.18243243243243243
Score pour une taille de fenêtre de 8: 0.18623310810810811
Score pour une taille de fenêtre de 9: 0.1891891891891892
Score pour une taille de fenêtre de 10: 0.19130067567567569
Score pour une taille de fenêtre de 11: 0.19045608108108109
Score pour une taille de fenêtre de 12: 0.19172297297297297
Score pour une taille de fenêtre de 13: 0.19341216216216217
Score pour une taille de fenêtre de 14: 0.19763513513513514
Score pour une taille de fenêtre de 15: 0.19763513513513514
Score pour une taille de fenêtre de 16: 0.19721283783783783
Score pour une taille de fenêtre de 17: 0.19

### Réponse à la question

Il semble que plus la fenêtre est grande, plus le score est élevé. Evidemment, le score ne s'amméliore plus si la taille de la fenêtre dépasse la taille de la phrase.

Ainsi, on trouve au mieux un score de 0,2010 pour senses1 et 0,2622 pour senses2. On trouve donc un meilleur score pour senses2. Il semble donc que donner plus de contexte aide à déterminer le sens du mot `interest` (ce qui est assez intuitif).

## 3. Utilisation de word2vec pour la similarité contexte vs. synset

**3a.** En réutilisant une partie du code de `wsd_lesk`, veuillez maintenant définir une fonction `wsd_word2vec(senses, sentence)` qui choisit le sens en utilisant la similarité **word2vec** étudiée dans le labo précédent. 
* Vous pouvez chercher dans la [documentation des KeyedVectors](https://radimrehurek.com/gensim/models/keyedvectors.html) comment calculer directement la similarité entre deux listes de mots.
* Comme `wsd_lesk`, la nouvelle fonction `wsd_word2vec` prend en argument une liste de listes de mots-clés par sens (comme `senses1` et `senses2` ci-dessus), et une phrase avec une occurrence annotée de *interest* ou *interests*.
* La fonction retourne le numéro du sens le plus probable selon la similarité word2vec entre les mots du sens et ceux du voisinage de *interest*.  En cas d'égalité, tirer le sens au sort.
* Vous pouvez régler la taille du voisinage (`window_size`) par l'expérimentation.

In [9]:
from gensim import downloader as api
w2v_vectors = api.load("word2vec-google-news-300")

In [10]:
import gensim
from gensim.models import KeyedVectors
path_to_model = "~/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz" # à adapter
wv_model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)  # C bin format

In [11]:
# Veuillez répondre ici à la question.
import random

def wsd_word2vec(senses, sentence, window_size):

    index = 0
    for pos, word in enumerate(sentence):
        if re.match('^interest[s]?_[1-6]$', word):
            index = pos
            break

    context = sentence[max(index - window_size, 0):min(index + window_size + 1, len(sentence))]
    similarities = []

    for sense in senses:
        similarities.append(wv_model.n_similarity(sense, context))

    max_score = 0
    max_index = []
    for j, sim in enumerate(similarities):
        if sim > max_score:
            max_score = sim
            max_index = [j]
        elif sim == max_score:
            max_index.append(j)

    return random.choice(max_index) + 1

**3b.** Appliquez maintenant la même méthode `evaluate_wsd` avec la fonction `wsd_word2vec` (en cherchant une bonne valeur de la taille de la fenêtre) et affichez le score de la similarité word2vec.  Comment se compare-t-il avec le score précédent (Lesk) ?

In [ ]:
# Veuillez répondre ici à la question.
print('Sense1')
for i in range(1, 20):
  print(f'Score pour une taille de fenêtre de {i}: {evaluate_wsd(wsd_word2vec, senses1, sentences, i)}')

print('\nSense2')
for i in range(1, 20):
  print(f'Score pour une taille de fenêtre de {i}: {evaluate_wsd(wsd_word2vec, senses2, sentences, i)}')

Sense1


### Réponse à la question

Le score maximal pour senses 1 est de 0.4268 pour un taille de fenêtre de 13 et celui de senses2 est 0.6152 pour une fenêtre de taille 6. On constate que, dans les deux cas, les scores sont bien meilleurs qu'avec Lesk. Cependant, on constate aussi qu'en utilisant word2vec et non Lesk, le score n'augmente pas forcément avec la taille de la fenêtre. Cela est dû à l'utilisation de la similarité cosinus pour déterminer la proximité entre les mots.

A nouveau, c'est senses2 qui a un score plus élevé. Cela semble confirmer l'hypothèse que donner plus de contexte aide à déterminer le sens du mot `interest`.

## 4. Classification supervisée avec des traits lexicaux
Vous entraînerez maintenant des classifieurs pour prédire le sens d'une occurrence dans une phrase.  Le premier but sera de transformer chaque phrase en un ensemble d'attributs pour formater les données en vue des expériences de classification.

Veuillez utiliser le classifieur `NaiveBayesClassifier` fourni par NLTK.  Le mode d'emploi se trouve dans le [Chapitre 6, sections 1.1-1.3](https://www.nltk.org/book/ch06.html) du livre NLTK.  Consultez-le attentivement pour trouver comment formater les données.  De plus, il faudra séparer les données en sous-ensembles d'entraînement et de test.

On vous propose de nommer les attributs `word-k`, ..., `word-2`, `word-1`, `word+1`, `word+2`, ..., `word+k` (fenêtre de taille `2*k` autour de *interest*).  Leurs valeurs sont les mots observés aux emplacements respectifs, ou `NONE` si la position dépasse l'étendue de la phrase.  Vous ajouterez un attribut nommé `word0` qui est l'occurrence du mot *interest* au singulier ou au pluriel.  

Pour chaque occurrence de *interest*, vous devrez donc créer la représentation suivante (où `6` est le numéro du sens, essentiel pour l'entraînement, mais à cacher lors de l'évaluation) :
```
[{'word-1': 'in', 'word+1': 'rates', 'word-2': 'declines', 'word+2': 'NONE', 'word0': 'interest'}, 6]
```

**4a.** En partant de la liste des phrases appelée `sentences` préparée plus haut, veuillez générer la liste avec toutes les représentation, appelée `items_with_features`.  Vous pouvez vous aider du livre NLTK.

In [31]:
# Veuillez répondre ici à la question.
k = 2
items_with_features = []
for sentence in sentences:
    for idx, word in enumerate(sentence):
        if re.match('^interest[s]?_[1-6]$', word):
            real_word, label = word.split("_")

            features = {}

            for offset in range(1, k + 1):
                features[f"word-{offset}"] = sentence[idx - offset] if idx - offset >= 0 else "NONE"
                features[f"word+{offset}"] = sentence[idx + offset] if idx + offset < len(sentence) else "NONE"

            features["word0"] = real_word

            items_with_features.append([features, int(label)])


print(f"Nombre de phrases: {len(items_with_features)}")
print(f"Extrait: {items_with_features[151:154]}")

Nombre de phrases: 2368
Extrait: [[{'word-1': 'investor', 'word+1': 'in', 'word-2': 'NONE', 'word+2': 'stock', 'word0': 'interest'}, 1], [{'word-1': 'western', 'word+1': 'to', 'word-2': 'the', 'word+2': 'see', 'word0': 'interest'}, 4], [{'word-1': 'of', 'word+1': ',', 'word-2': 'expression', 'word+2': "''", 'word0': 'interest'}, 1]]


**4b.** Veuillez séparer les données aléatoirement en 80% pour l'entraînement et 20%  pour l'évaluation.  Veuillez faire une division stratifiée : les deux sous-ensembles doivent contenir les mêmes proportions de sens que l'ensemble de départ.  Ils seront appelés `iwf_train` et `iwf_test`.

In [32]:
from random import shuffle

In [33]:
iwf_train = []
iwf_test  = []
# Veuillez répondre ici à la question.
items_by_features = {1: [], 2: [], 3: [], 4: [], 5: [], 6: []}
for item in items_with_features:
    items_by_features[item[1]].append(item[0])

for i, sent in items_by_features.items():
    shuffle(sent)
    split = int(len(sent) * 0.8)
    iwf_train.extend([sent, i] for sent in sent[:split])
    iwf_test.extend([sent, i] for sent in sent[split:])

print(f"Taille de train: {len(iwf_train)}, Taille de test: {len(iwf_test)}")
print(f"Extrait: {iwf_test[:2], iwf_test[-2:]}")

Taille de train: 1891, Taille de test: 477
Extrait: ([[{'word-1': 'its', 'word+1': 'earlier', 'word-2': 'announced', 'word+2': 'this', 'word0': 'interest'}, 1], [{'word-1': 'less', 'word+1': 'than', 'word-2': 'attracting', 'word+2': 'some', 'word0': 'interest'}, 1]], [[{'word-1': 'with', 'word+1': 'rates', 'word-2': 'NONE', 'word+2': 'rising', 'word0': 'interest'}, 6], [{'word-1': 'in', 'word+1': 'rates', 'word-2': 'drop', 'word+2': ',', 'word0': 'interest'}, 6]])


**4c.** Veuillez créer une instance de `NaiveBayesClassifier`, l'entraîner sur `iwf_train` et la tester sur `iwf_test` (voir la documentation NLTK).  En expérimentant avec différentes largeurs de fenêtres, quel est le meilleur score que vous obtenez (avec la fonction `accuracy` de NLTK) sur l'ensemble de test ?  Comment se compare-t-il avec les précédents ?

In [34]:
from nltk.classify import naivebayes
# Veuillez répondre ici à la question.
classifier = naivebayes.NaiveBayesClassifier.train(iwf_train)

print(f"Score de l'algorithme de classification Naive Bayes: {nltk.classify.accuracy(classifier, iwf_test)}")

Score de l'algorithme de classification Naive Bayes: 0.870020964360587


### Réponse à la question

Avec un fenêtre de taille 2, nous obtenons un score d'environ 88%, qui varie un peu en fonction de la répartition aléatoire des données entre la dataset de train et celui de test. En testant avec différentes valeurs de taille de fenêtre, on peut constater que le score varie peu.

Ce score est largement meilleur que les scores obtenus avec les algorithmes de Lesk (\~+65%) et word2vec (\~+40-20%).



**4d.** En utilisant la fonction `show_most_informative_features()`, veuillez afficher les attributs les plus informatifs et commenter le résultat.

In [35]:
classifier.show_most_informative_features()

Most Informative Features
                   word0 = 'interests'         3 : 1      =     99.2 : 1.0
                  word+1 = 'in'                1 : 6      =     65.7 : 1.0
                  word-1 = 'other'             3 : 6      =     45.2 : 1.0
                  word+1 = 'of'                4 : 6      =     37.4 : 1.0
                  word+2 = '.'                 6 : 1      =     37.3 : 1.0
                  word+2 = ','                 6 : 5      =     28.8 : 1.0
                  word-2 = 'have'              1 : 6      =     24.1 : 1.0
                  word+2 = 'on'                6 : 5      =     19.5 : 1.0
                  word-1 = 'in'                6 : 5      =     17.6 : 1.0
                  word-1 = 'own'               4 : 6      =     17.3 : 1.0


### Réponse à la question
La sortie se lit comme suit: emplacement:attribut label1:label2 chance qu'a le mot d'appartenir au label1 plutôt qu'au label2.

On peut constater que les attributs les plus informatifs sont:
- savoir si interest est au singulier ou au pluriel
- savoir si interest est suivi de `in` ou `of`
- savoir si interest est précédé de `other`
- savoir si interest est 2 crans avant un signe de ponctuation `.` ou `,`.

Cette observation s'explique par le fait que certains contextes linguistiques ou syntaxiques sont fortement corrélés à des sens spécifiques du mot interest. Voici des raison possibles pour chaque attribut informatif :
- Singulier ou pluriel : Le fait que interest soit au singulier ou au pluriel (interests) peut indiquer un sens différent. Par exemple, interests est souvent utilisé pour parler de parts dans une entreprise (sens 5) ou d'intérêts multiples (sens 3).
- Suivi de in ou of : Les prépositions qui suivent interest influencent son sens. Par exemple :
    -   interest in est souvent lié à l'attention ou à la curiosité (sens 1 ou 3).
    -   interest of peut être lié à un avantage ou un bénéfice (sens 4).
- Précédé de other : L'expression other interests est souvent utilisée pour parler de passe-temps ou d'activités (sens 3).
- Proximité avec une ponctuation (. ou ,) : La ponctuation peut indiquer la fin d'une phrase ou une séparation dans une phrase, ce qui peut aider à délimiter le contexte immédiat de interest.

**4e.** On souhaite également obtenir les scores pour chaque sens.  Pour ce faire, il faut demander les prédictions une par une au classifieur (voir le [livre NLTK](https://www.nltk.org/book/ch06.html)), et comptabiliser les prédictions correctes pour chaque sens.  Vous pouvez vous inspirer de `evaluate_wsd`, et écrire une fonction `evaluate_wsd_supervised(classifier, items_with_features)`, que vous appliquerez aux donnés `iwf_test`.  Veuillez afficher ces scores.

In [36]:
def evaluate_wsd_supervised(classifier, items_with_features):
    scores = {}
    total_counts = {}

    for item in items_with_features:
        features, true_label = item
        predicted_label = classifier.classify(features)

        if true_label not in scores:
            scores[true_label] = 0
            total_counts[true_label] = 0

        total_counts[true_label] += 1

        if predicted_label == true_label:
            scores[true_label] += 1

    print("Scores par sens :")
    for sense in scores:
        correct_count = scores[sense]
        total_count = total_counts[sense]
        accuracy = correct_count / total_count if total_count > 0 else 0
        print(f"Sens {sense} : {correct_count}/{total_count} corrects ({accuracy:.2%})")

    return scores

evaluate_wsd_supervised(classifier, iwf_test)

Scores par sens :
Sens 1 : 57/73 corrects (78.08%)
Sens 2 : 0/3 corrects (0.00%)
Sens 3 : 5/14 corrects (35.71%)
Sens 4 : 28/36 corrects (77.78%)
Sens 5 : 83/100 corrects (83.00%)
Sens 6 : 242/251 corrects (96.41%)


{1: 57, 2: 0, 3: 5, 4: 28, 5: 83, 6: 242}

## 5. Conclusion

Veuillez recopier ci-dessous, en guise de conclusion, les scores des trois expériences réalisées, pour pouvoir les comparer d'un coup d'oeil.  Quel est le meilleur score obtenu?

Voic un tableau contenant le récapitulatif des scores obtenus pour les trois méthodes de désambiguïsation lexicale :

|                    Méthode                     |                    score senses1                     |score senses2|
|:----------------------------------------------:|:----------------------------------------------------:|:-----------:|
|          Algorithme de Lesk simplifié          |                        0.2010                        |0.2622|
|                    Word2Vec                    |                        0.4268                        |0.6152|
| Classification supervisée avec traits lexicaux | \~ 0.88 (varie un peu selon répartition train/ test) | - |

Le meilleure score obtenu est celui de la classification supervisée avec traits lexicaux, qui est de l'ordre de 88%. On peut donc conclure que la classification supervisée avec traits lexicaux est la méthode la plus efficace pour la désambiguïsation lexicale, dans ce cas.

## Fin du laboratoire

Merci de nettoyer votre feuille, sauvegarder le résultat, et soumettre le *notebook* sur Cyberlearn.